In [152]:
from BPnumba.Individual import Ind,CodeSolution
from BPnumba.BPPH import create_Bin,DBLF,Bin
from BPnumba.Poblation import EvalPob
from BPmodule.BPGen import GetInstance
from BPnumba.Poblation import CreateRotHPob
from BPnumba.BoxN import InstaceBoxes
from numba.typed import List as NumbaList
import time


In [160]:
P5A2 = GetInstance("P5A2")
UBOXES = InstaceBoxes(P5A2[1][10])
rot = 0
Pob = CreateRotHPob(pop =100, ITEMS = UBOXES.copy(), rotType = rot)
bin : Bin  = create_Bin(NumbaList(P5A2[0]))

In [161]:
start_time = time.time()
EvalPob(Pob, UBOXES,NumbaList(P5A2[0]), rot)
print("--- %s seconds ---" % (time.time() - start_time))

--- 16.14203453063965 seconds ---


In [162]:
ind:Ind =Pob[99]


In [163]:
ind.fi, ind.genome, ind.genome_r

(0.7113988888888889,
 ListType[int64]([64, 112, 1, 41, 33, 67, 54, 3, 75, 79, 58, 87, 61, 98, 18, 70, 52, 55, 69, 86, 32, 141, 72, 24, 37, 120, 96, 9, 113, 130, 89, 139, 78, 91, 39, 93, 42, 63, 142, 140, 20, 46, 150, 102, 143, 126, 114, 57, 81, 12, 108, 6, 23, 25, 34, 35, 105, 115, 137, 26, 13, 4, 103, 74, 84, 128, 80, 135, 30, 31, 121, 148, 97, 127, 118, 53, 132, 38, 110, 124, 8, 21, 66, 40, 82, 83, 145, 133, 14, 131, 77, 43, 5, 76, 125, 27, 29, 59, 111, 51, 7, 106, 129, 122, 94, 144, 147, 151, 123, 28, 22, 47, 48, 100, 68, 117, 19, 71, 90, 107, 104, 17, 49, 101, 73, 60, 138, 95, 62, 109, 134, 88, 16, 146, 119, 11, 15, 65, 50, 136, 2, 36, 56, 45, 116, 149, 85, 44, 10, 99, 92, ...]),
 ListType[int64]([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [164]:
ind.fi = bin.getLoadVol() / (bin.dimensions[0]*bin.dimensions[1]*bin.dimensions[2])
ind.codeSolution = CodeSolution(bin.getBoxes())
ind.fi, ind.codeSolution 

(0.0, '|')

In [165]:
bin.getLoadVol() , bin.dimensions[0]*bin.dimensions[1]*bin.dimensions[2]

(0, 27000000)

In [166]:
#path=r"A:/Documents/Unity/BPP/Assets/bpp.json"
#path2=r"A:/Documents/Unity/BPP/Assets/points.json"
#ExpotarJSON(path=path,rot=rot,Cont=bin,Data=UBOXES)
#PossiblePositions(path=path2,positions=bin.extrapts)

In [170]:
gen = [12,5,13,4,6,7,1,2]

gen[:1]

[12]